In [1]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix, triu, tril, hstack
import graphrelations as gr
from iterativemethods import *
from aggregation_coarsening import *
from multilevelmethod import *


In [2]:
testmat = np.zeros((8,8))
is_edge = [(1,5),(2,4), (2,8), (3,5),(4,2), (4,6), (5,1), (5,3), (5,6), (5,7), (6,4), (6,5), (6,8), (7,5), (8,2), (8,6)]
for i,j in is_edge:
    testmat[i-1,j-1] = 1
testmat = coo_matrix(testmat)
ev = gr.EV(testmat)
ee = gr.EE(ev)

In [3]:
# testweights = [0.1,5.0,1.2,3.0,1.75,2.0,4.0,3.0]

In [4]:
weights = edge_modularity_weights(testmat)
max_edges = luby(ee, weights)
P = vert_agg(max_edges, ev)

In [5]:
print(P.toarray())

[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


In [6]:
AC_1 = P.T @ testmat @ P
print(AC_1.toarray())
#this is where you check if cfactor is reached. for cfactor = 1.5, we can do one more step
# if 8/1.5=5.3333 >= numverts in AC_n, stop.

[[2. 0. 1. 1. 1. 0.]
 [0. 2. 0. 1. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0.]]


In [7]:
lala = coo(AC_1 - AC_1.diagonal())
print(lala.toarray())

[[ 0. -2.  1.  1.  1.  0.]
 [-2.  0.  0.  1.  0.  1.]
 [-1. -2.  0.  0.  0.  0.]
 [-1. -1.  0.  0.  0.  1.]
 [-1. -2.  0.  0.  0.  0.]
 [-2. -1.  0.  1.  0.  0.]]


In [8]:
ev2 = gr.EV(AC_1)
print(ev2.toarray())

[[1 0 1 0 0 0]
 [0 1 0 1 0 0]
 [1 0 0 1 0 0]
 [1 0 0 0 1 0]
 [0 1 0 0 0 1]
 [0 0 0 1 0 1]]


In [9]:
ev2 = gr.EV(AC_1)
ee2 = gr.EE(ev2)
md = mod_mat_inefficient(AC_1)
print(ee2.toarray())
print(md)

[[0 0 1 1 0 0]
 [0 0 1 0 1 1]
 [1 1 0 1 0 1]
 [1 0 1 0 0 0]
 [0 1 0 0 0 1]
 [0 1 1 0 1 0]]
[[ 0.4375 -1.25    0.6875  0.0625  0.6875 -0.625 ]
 [-1.25    1.     -0.25    0.25   -0.25    0.5   ]
 [ 0.6875 -0.25   -0.0625 -0.1875 -0.0625 -0.125 ]
 [ 0.0625  0.25   -0.1875 -0.5625 -0.1875  0.625 ]
 [ 0.6875 -0.25   -0.0625 -0.1875 -0.0625 -0.125 ]
 [-0.625   0.5    -0.125   0.625  -0.125  -0.25  ]]


In [10]:
weights2 = edge_modularity_weights(AC_1)
print(AC_1.data)
print(ee2.toarray())
print(weights2)
max_edges2 = luby(ee2, weights2)
P2 = vert_agg(max_edges2, ev2)
print(P2.toarray())

[1. 1. 1. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
[[0 0 1 1 0 0]
 [0 0 1 0 1 1]
 [1 1 0 1 0 1]
 [1 0 1 0 0 0]
 [0 1 0 0 0 1]
 [0 1 1 0 1 0]]
[0.6875 0.25   0.0625 0.6875 0.5    0.625 ]
[[1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


In [11]:
AC_2 = P2.T @AC_1 @P2
print(AC_2.toarray())
#now cfactor is reached, stop and compose the P's

[[4. 0. 1. 1. 0.]
 [0. 2. 1. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0.]]


In [12]:
Pc = P @ P2
print(Pc.toarray())
AC_final = Pc.T @ testmat @ Pc
print(AC_final.toarray())

[[1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
[[4. 0. 1. 1. 0.]
 [0. 2. 1. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0.]]


In [13]:
q = P_coarse(testmat, 1.5, modularity_weights=True)

In [14]:
print(q[0].toarray())
print(q[1].toarray())


[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
[[4. 0. 1. 1. 0.]
 [0. 2. 1. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0.]]


In [17]:
w_la = edge_modularity_weights(mod_la)

[-1. -1. -1. -1. -1. -1.]


In [18]:
P_coarse_LA, A_coarse_LA = P_coarse(mod_la, modularity_weights=True)

In [19]:
print(P_coarse_LA.toarray())
print(A_coarse_LA.toarray())

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[ 3.  0. -1. -1. -1.  0.]
 [ 0.  2.  0. -1.  0.  0.]
 [-1.  0.  1.  0.  0.  0.]
 [-1. -1.  0.  3.  0.  0.]
 [-1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  2.]]


In [22]:
"""
here's where I will test my two level method
"""
x = B_TL_symmetric(testmat,c_factor=1.5)
print(x)

0.8964456940287685
0.6484832113812992
[3.31933204 0.66298717 4.07407825 1.23061722 3.12109682 1.5783035
 3.51974366 0.29702732]
